## Intro to deep learning for medical imaging 

[MD.ai](https://www.md.ai)

## Lesson 3. RSNA Pneumonia Detection Challenge (Kaggel API)

The [Radiological Society of North America](http://www.rsna.org/) Pneumonia Detection Challenge: https://www.kaggle.com/c/rsna-pneumonia-detection-challenge

This notebook covers the basics of parsing the competition dataset, training using a detector basd on the [Mask-RCNN algorithm](https://arxiv.org/abs/1703.06870) for object detection and instance segmentation.  

This notebook is developed by [MD.ai](https://www.md.ai), the platform for medical AI. 
This notebook requires Google's [TensorFlow](https://www.tensorflow.org/) machine learning framework.

**Intro to deep learning for medical imaging lessons**

- Lesson 1. Classification of chest vs. adominal X-rays using TensorFlow/Keras [Github](https://github.com/mdai/ml-lessons/blob/master/lesson1-xray-images-classification.ipynb) [Annotator](https://public.md.ai/annotator/project/PVq9raBJ)

- Lesson 2. Lung X-Rays Semantic Segmentation using UNets. [Github](https://github.com/mdai/ml-lessons/blob/master/lesson2-lung-xrays-segmentation.ipynb)
[Annotator](https://public.md.ai/annotator/project/aGq4k6NW/workspace) 

- Lesson 3. RSNA Pneumonia detection using Kaggle data format [Github](https://github.com/mdai/ml-lessons/blob/master/lesson3-rsna-pneumonia-detection-kaggle.ipynb) [Annotator](https://public.md.ai/annotator/project/LxR6zdR2/workspace) 
  
- Lesson 3. RSNA Pneumonia detection using MD.ai python client library [Github](https://github.com/mdai/ml-lessons/blob/master/lesson3-rsna-pneumonia-detection-mdai-client-lib.ipynb) [Annotator](https://public.md.ai/annotator/project/LxR6zdR2/workspace) 

In [1]:
import os
import sys

import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
import pydicom
from imgaug import augmenters as iaa
from tqdm import tqdm
import pandas as pd
import glob
import scipy.signal as signal
import skimage.morphology as sm

# enter your Kaggle credentionals here
os.environ['KAGGLE_USERNAME']="lixiaoqi"
os.environ['KAGGLE_KEY']="a96b56107f574e1cf4e14e3822ca2c93"

os.chdir('E:\学习\ml-lessons')
# Root directory of the project
ROOT_DIR = os.path.abspath('./lesson3-data')

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, 'logs')

#if not os.path.exists(ROOT_DIR):
#    os.makedirs(ROOT_DIR)
os.chdir(ROOT_DIR)####

# Import Mask RCNN
sys.path.append(os.path.join(ROOT_DIR, 'Mask_RCNN'))  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

train_dicom_dir = os.path.join(ROOT_DIR, 'stage_1_train_images')
test_dicom_dir = os.path.join(ROOT_DIR, 'stage_1_test_images')

'''
- dicom_fps is a list of the dicom image path and filenames 
- image_annotions is a dictionary of the annotations keyed by the filenames
- parsing the dataset returns a list of the image filenames and the annotations dictionary
'''
# 将图片的绝对路径以list方式保存起来，这样就可以通过索引的方式直接获取图像路径及名称
def get_dicom_fps(dicom_dir):
    dicom_fps = glob.glob(dicom_dir+'/'+'*.dcm')
    return list(set(dicom_fps))

#image_fps保存了list类型的图像路径及名称，image_annotations保存了每一张图像所对应的识别框的信息
def parse_dataset(dicom_dir, anns): 
    image_fps = get_dicom_fps(dicom_dir)
    image_annotations = {fp: [] for fp in image_fps}
    for index, row in anns.iterrows(): 
        fp = os.path.join(dicom_dir, row['patientId']+'.dcm')
        image_annotations[fp].append(row)
    return image_fps, image_annotations 

# set color for class
def get_colors_for_class_ids(class_ids):
    colors = []
    for class_id in class_ids:
        if class_id == 1:
            colors.append((.941, .204, .204))
    return colors

#  直方图均衡
def histep(img, nbr_bins=256):
    """Histogram equalization of a grayscale image."""
    # 获取直方图p(r)
    imhist, bins = np.histogram(img.flatten(), nbr_bins, normed=True)
    
    # 获取T(r)
    cdf = imhist.cumsum()  # cumulative distribution function
    cdf = 255 * cdf / cdf[-1]
    
    #获取s, 并用s替换原始图像对应的灰度值
    result = np.interp(img.flatten(), bins[:-1], cdf)
    return result.reshape(img.shape), cdf

# The following parameters have been selected to reduce running time for demonstration purposes 
# These are not optimal 
# 为在RSNA pneumonia数据集上训练模型而重写mask_rcnn中的Config类
class DetectorConfig(Config):
    # 给该configuration取一个可识别的名字
    NAME = 'pneumonia'
    
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 4#4,8
    #BATCH_SIZE是上面两个值的乘积
    BACKBONE = 'resnet50'
    
    NUM_CLASSES = 2   # background + 1 pneumonia classes
    
    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 512  ####64,256,512
    IMAGE_MAX_DIM = 512
    
    RPN_ANCHOR_SCALES = (32, 64, 128, 256)#(16, 32, 64, 128)   #(32, 64, 128, 256)
    
    TRAIN_ROIS_PER_IMAGE = 32  #16,32
    
    MAX_GT_INSTANCES = 4
    
    DETECTION_MAX_INSTANCES = 3
    DETECTION_MIN_CONFIDENCE = 0.7   #待定
    DETECTION_NMS_THRESHOLD = 0.01
    
    # RPN_TRAIN_ANCHORS_PER_IMAGE = 16
    STEPS_PER_EPOCH = 400  #100
    # TOP_DOWN_PYRAMID_SIZE = 32
    
config = DetectorConfig()
#config.display()

class DetectorDataset(utils.Dataset):
    """Dataset class for training pneumonia detection on the RSNA pneumonia dataset.
    """

    def __init__(self, image_fps, image_annotations, orig_height, orig_width):
        super().__init__(self)
        
        # 添加类
        self.add_class('pneumonia', 1, 'Lung Opacity')
   
        # 添加图像
        for i, fp in enumerate(image_fps):
            annotations = image_annotations[fp]
            self.add_image('pneumonia', image_id=i, path=fp, 
                           annotations=annotations, orig_height=orig_height, orig_width=orig_width)
            
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

    def load_image(self, image_id):
        info = self.image_info[image_id]
        fp = info['path']
        ds = pydicom.read_file(fp)
        image = ds.pixel_array
        
        # image = signal.medfilt(image, kernel_size=5)   # 对图片进行中值滤波
        # image = image.astype(np.uint8)
        # image = cv2.equalizeHist(image)  # 对图片进行直方图均衡处理
        #image = sm.closing(image,sm.disk(5))  #用边长为5的圆形滤波器进行闭运算滤波
        
        image, _ = histep(image)   # 对图片进行直方图均衡处理
        # If grayscale. Convert to RGB for consistency.
        if len(image.shape) != 3 or image.shape[2] != 3:
            image = np.stack((image,) * 3, -1)
        return image

    def load_mask(self, image_id):
        info = self.image_info[image_id]
        annotations = info['annotations']
        count = len(annotations)
        if count == 0:
            mask = np.zeros((info['orig_height'], info['orig_width'], 1), dtype=np.uint8)
            class_ids = np.zeros((1,), dtype=np.int32)
        else:
            mask = np.zeros((info['orig_height'], info['orig_width'], count), dtype=np.uint8)
            class_ids = np.zeros((count,), dtype=np.int32)
            for i, a in enumerate(annotations):
                if a['Target'] == 1:
                    x = int(a['x'])
                    y = int(a['y'])
                    w = int(a['width'])
                    h = int(a['height'])
                    mask_instance = mask[:, :, i].copy()
                    cv2.rectangle(mask_instance, (x, y), (x+w, y+h), 255, -1)
                    mask[:, :, i] = mask_instance
                    class_ids[i] = 1
        return mask.astype(np.bool), class_ids.astype(np.int32)

    
# 训练集
f=open(os.path.join(ROOT_DIR, 'stage_1_train_labels.csv'))
anns = pd.read_csv(f)
image_fps, image_annotations = parse_dataset(train_dicom_dir, anns=anns)
# Original DICOM image size: 1024 x 1024
ORIG_SIZE = 1024

# Modify this line to use more or fewer images for training/validation. 
# To use all images, do: image_fps_list = list(image_fps)
image_fps_list = list(image_fps)#[:1000] 
# 将数据分为训练集和验证集
# split ratio is set to 0.9 vs. 0.1 (train vs. validation, respectively)
#sorted(image_fps_list)
random.seed(42)
random.shuffle(image_fps_list)

# 训练集和验证集
val_size = 1500
image_fps_val = image_fps_list[:val_size]
image_fps_train = image_fps_list[val_size:]
'''
validation_split = 0.1
split_index = int((1 - validation_split) * len(image_fps_list))
image_fps_train = image_fps_list[:split_index]
image_fps_val = image_fps_list[split_index:]
'''
print('训练和测试样例个数:',len(image_fps_train), len(image_fps_val))

# prepare the training dataset
dataset_train = DetectorDataset(image_fps_train, image_annotations, ORIG_SIZE, ORIG_SIZE)
dataset_train.prepare()

# Show annotation(s) for a DICOM image 
# choice() 方法返回一个列表，元组或字符串的随机项。
# test_fp = random.choice(image_fps_train)
# print(image_annotations[test_fp])

# prepare the validation dataset
dataset_val = DetectorDataset(image_fps_val, image_annotations, ORIG_SIZE, ORIG_SIZE)
dataset_val.prepare()

model = modellib.MaskRCNN(mode='training', config=config, model_dir=MODEL_DIR)

# Image augmentation (light but constant)
'''augmentation = iaa.Sequential([
        iaa.Fliplr(0.5),
        iaa.Affine(
            scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
            translate_percent={"x": (-0.04, 0.04), "y": (-0.08, 0.08)},
            rotate=(-5, 5),
            shear=(-1, 1)
        )
    
])'''
augmentation = iaa.Sequential([
        iaa.Fliplr(0.5),
        iaa.Affine(
                scale={"x": (0.98, 1.02), "y": (0.98, 1.02)},
                translate_percent={"x": (-0.02, 0.02), "y": (-0.04, 0.04)},
                rotate=(-2, 2),
                shear=(-1, 1),
            ),
    
])
'''augmentation = iaa.Sequential([
        iaa.Fliplr(0.5),
        iaa.OneOf([  # geometric transform
            iaa.Affine(
                scale={"x": (0.98, 1.02), "y": (0.98, 1.02)},
                translate_percent={"x": (-0.02, 0.02), "y": (-0.04, 0.04)},
                rotate=(-2, 2),
                shear=(-1, 1),
            ),
            #iaa.PiecewiseAffine(scale=(0.001, 0.025)),
        ]),
        iaa.OneOf([  # brightness or contrast
            iaa.Multiply((0.9, 1.1)),
            iaa.ContrastNormalization((0.9, 1.1)),
        ]),
        iaa.OneOf([  # blur or sharpen
            iaa.GaussianBlur(sigma=(0.0, 0.1)),
            iaa.Sharpen(alpha=(0.0, 0.1)),
        ])
])'''
'''
# 加载预训练的COCO权重
COCO_MODEL_PATH = "./Mask_RCNN/samples/coco/mask_rcnn_coco.h5"

#设定权重的初始化方式，有imagenet，coco,last三种
init_with = "coco"
if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    model.load_weights(model.find_last()[1], by_name=True)
'''
#'E:\学习\ml-lessons\lesson3-data\logs\mask_rcnn_pneumonia_0037.h5'
model_path = 'E:\学习\ml-lessons\lesson3-data\logs\pneumonia20181026T2247\mask_rcnn_pneumonia_0016.h5'
model.load_weights(model_path, by_name=True)

NUM_EPOCHS = 35
LEARNING_RATE = 0.005
# Train Mask-RCNN Model 
import warnings 
warnings.filterwarnings("ignore")
'''
# 头两个epoch使用较大的learning rate，并且只训练heads部分
model.train(dataset_train, dataset_val, 
            learning_rate=LEARNING_RATE*2, 
            epochs=2, 
            layers='heads',
            augmentation=None)  # no need to augment yet
model.train(dataset_train, dataset_val, 
            learning_rate=LEARNING_RATE, 
            epochs=6, 
            layers='all',
            augmentation=augmentation)
# 后面的epoch使用更小的learning rate
model.train(dataset_train, dataset_val, 
            learning_rate=LEARNING_RATE/5, 
            epochs=16, 
            layers='all',
            augmentation=augmentation)'''
model.train(dataset_train, dataset_val, 
            learning_rate=LEARNING_RATE/50, 
            epochs=NUM_EPOCHS,
            layers='all',
            augmentation=augmentation)
'''
model.train(dataset_train, dataset_val, 
            learning_rate=LEARNING_RATE/500, 
            epochs=20,
            layers='all',
            augmentation=augmentation)
'''
print('train complete')

Using TensorFlow backend.


训练和测试样例个数: 25184 1500
Re-starting from epoch 16

Starting at epoch 16. LR=0.0001

Checkpoint Path: E:\学习\ml-lessons\lesson3-data\logs\pneumonia20181026T2247\mask_rcnn_pneumonia_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a

 53/400 [==>...........................] - ETA: 2:29:58 - loss: 1.0444 - rpn_class_loss: 0.0130 - rpn_bbox_loss: 0.1830 - mrcnn_class_loss: 0.1719 - mrcnn_bbox_loss: 0.3443 - mrcnn_mask_loss: 0.33 - ETA: 1:27:07 - loss: 0.9869 - rpn_class_loss: 0.0083 - rpn_bbox_loss: 0.1818 - mrcnn_class_loss: 0.1682 - mrcnn_bbox_loss: 0.2941 - mrcnn_mask_loss: 0.33 - ETA: 1:06:48 - loss: 1.0499 - rpn_class_loss: 0.0080 - rpn_bbox_loss: 0.1976 - mrcnn_class_loss: 0.1843 - mrcnn_bbox_loss: 0.3211 - mrcnn_mask_loss: 0.33 - ETA: 55:58 - loss: 1.0625 - rpn_class_loss: 0.0073 - rpn_bbox_loss: 0.1970 - mrcnn_class_loss: 0.1896 - mrcnn_bbox_loss: 0.3396 - mrcnn_mask_loss: 0.3290 - ETA: 52:25 - loss: 1.1621 - rpn_class_loss: 0.0117 - rpn_bbox_loss: 0.2362 - mrcnn_class_loss: 0.2161 - mrcnn_bbox_loss: 0.3530 - mrcnn_mask_loss: 0.34 - ETA: 46:11 - loss: 1.1935 - rpn_class_loss: 0.0167 - rpn_bbox_loss: 0.2449 - mrcnn_class_loss: 0.2363 - mrcnn_bbox_loss: 0.3569 - mrcnn_mask_loss: 0.33 - ETA: 42:50 - loss: 1.1465

106/400 [======>.......................] - ETA: 27:08 - loss: 1.1840 - rpn_class_loss: 0.0106 - rpn_bbox_loss: 0.2424 - mrcnn_class_loss: 0.2342 - mrcnn_bbox_loss: 0.3506 - mrcnn_mask_loss: 0.34 - ETA: 26:48 - loss: 1.1813 - rpn_class_loss: 0.0106 - rpn_bbox_loss: 0.2423 - mrcnn_class_loss: 0.2330 - mrcnn_bbox_loss: 0.3497 - mrcnn_mask_loss: 0.34 - ETA: 26:38 - loss: 1.1780 - rpn_class_loss: 0.0105 - rpn_bbox_loss: 0.2412 - mrcnn_class_loss: 0.2335 - mrcnn_bbox_loss: 0.3479 - mrcnn_mask_loss: 0.34 - ETA: 26:38 - loss: 1.1780 - rpn_class_loss: 0.0105 - rpn_bbox_loss: 0.2403 - mrcnn_class_loss: 0.2338 - mrcnn_bbox_loss: 0.3479 - mrcnn_mask_loss: 0.34 - ETA: 26:28 - loss: 1.1791 - rpn_class_loss: 0.0105 - rpn_bbox_loss: 0.2421 - mrcnn_class_loss: 0.2326 - mrcnn_bbox_loss: 0.3479 - mrcnn_mask_loss: 0.34 - ETA: 26:34 - loss: 1.1792 - rpn_class_loss: 0.0104 - rpn_bbox_loss: 0.2416 - mrcnn_class_loss: 0.2327 - mrcnn_bbox_loss: 0.3487 - mrcnn_mask_loss: 0.34 - ETA: 26:23 - loss: 1.1779 - rpn_c

159/400 [==========>...................] - ETA: 21:08 - loss: 1.2065 - rpn_class_loss: 0.0102 - rpn_bbox_loss: 0.2645 - mrcnn_class_loss: 0.2292 - mrcnn_bbox_loss: 0.3503 - mrcnn_mask_loss: 0.35 - ETA: 21:00 - loss: 1.2067 - rpn_class_loss: 0.0103 - rpn_bbox_loss: 0.2646 - mrcnn_class_loss: 0.2294 - mrcnn_bbox_loss: 0.3504 - mrcnn_mask_loss: 0.35 - ETA: 21:01 - loss: 1.2069 - rpn_class_loss: 0.0102 - rpn_bbox_loss: 0.2642 - mrcnn_class_loss: 0.2292 - mrcnn_bbox_loss: 0.3511 - mrcnn_mask_loss: 0.35 - ETA: 20:59 - loss: 1.2096 - rpn_class_loss: 0.0103 - rpn_bbox_loss: 0.2652 - mrcnn_class_loss: 0.2291 - mrcnn_bbox_loss: 0.3519 - mrcnn_mask_loss: 0.35 - ETA: 20:53 - loss: 1.2080 - rpn_class_loss: 0.0103 - rpn_bbox_loss: 0.2640 - mrcnn_class_loss: 0.2285 - mrcnn_bbox_loss: 0.3519 - mrcnn_mask_loss: 0.35 - ETA: 20:52 - loss: 1.2090 - rpn_class_loss: 0.0105 - rpn_bbox_loss: 0.2642 - mrcnn_class_loss: 0.2290 - mrcnn_bbox_loss: 0.3519 - mrcnn_mask_loss: 0.35 - ETA: 20:46 - loss: 1.2113 - rpn_c

...

212/400 [==============>...............] - ETA: 16:50 - loss: 1.1316 - rpn_class_loss: 0.0083 - rpn_bbox_loss: 0.2478 - mrcnn_class_loss: 0.1844 - mrcnn_bbox_loss: 0.3353 - mrcnn_mask_loss: 0.35 - ETA: 16:45 - loss: 1.1330 - rpn_class_loss: 0.0083 - rpn_bbox_loss: 0.2480 - mrcnn_class_loss: 0.1845 - mrcnn_bbox_loss: 0.3361 - mrcnn_mask_loss: 0.35 - ETA: 16:43 - loss: 1.1323 - rpn_class_loss: 0.0083 - rpn_bbox_loss: 0.2482 - mrcnn_class_loss: 0.1843 - mrcnn_bbox_loss: 0.3358 - mrcnn_mask_loss: 0.35 - ETA: 16:38 - loss: 1.1310 - rpn_class_loss: 0.0083 - rpn_bbox_loss: 0.2480 - mrcnn_class_loss: 0.1839 - mrcnn_bbox_loss: 0.3354 - mrcnn_mask_loss: 0.35 - ETA: 16:35 - loss: 1.1307 - rpn_class_loss: 0.0083 - rpn_bbox_loss: 0.2480 - mrcnn_class_loss: 0.1840 - mrcnn_bbox_loss: 0.3351 - mrcnn_mask_loss: 0.35 - ETA: 16:34 - loss: 1.1297 - rpn_class_loss: 0.0082 - rpn_bbox_loss: 0.2482 - mrcnn_class_loss: 0.1838 - mrcnn_bbox_loss: 0.3346 - mrcnn_mask_loss: 0.35 - ETA: 16:30 - loss: 1.1314 - rpn_c

265/400 [==================>...........] - ETA: 12:58 - loss: 1.1307 - rpn_class_loss: 0.0082 - rpn_bbox_loss: 0.2525 - mrcnn_class_loss: 0.1814 - mrcnn_bbox_loss: 0.3332 - mrcnn_mask_loss: 0.35 - ETA: 12:53 - loss: 1.1303 - rpn_class_loss: 0.0082 - rpn_bbox_loss: 0.2520 - mrcnn_class_loss: 0.1817 - mrcnn_bbox_loss: 0.3332 - mrcnn_mask_loss: 0.35 - ETA: 12:51 - loss: 1.1299 - rpn_class_loss: 0.0082 - rpn_bbox_loss: 0.2516 - mrcnn_class_loss: 0.1818 - mrcnn_bbox_loss: 0.3333 - mrcnn_mask_loss: 0.35 - ETA: 12:47 - loss: 1.1322 - rpn_class_loss: 0.0083 - rpn_bbox_loss: 0.2538 - mrcnn_class_loss: 0.1820 - mrcnn_bbox_loss: 0.3332 - mrcnn_mask_loss: 0.35 - ETA: 12:43 - loss: 1.1333 - rpn_class_loss: 0.0083 - rpn_bbox_loss: 0.2542 - mrcnn_class_loss: 0.1824 - mrcnn_bbox_loss: 0.3334 - mrcnn_mask_loss: 0.35 - ETA: 12:40 - loss: 1.1351 - rpn_class_loss: 0.0083 - rpn_bbox_loss: 0.2546 - mrcnn_class_loss: 0.1830 - mrcnn_bbox_loss: 0.3337 - mrcnn_mask_loss: 0.35 - ETA: 12:35 - loss: 1.1348 - rpn_c

318/400 [======================>.......] - ETA: 9:16 - loss: 1.1333 - rpn_class_loss: 0.0084 - rpn_bbox_loss: 0.2565 - mrcnn_class_loss: 0.1793 - mrcnn_bbox_loss: 0.3333 - mrcnn_mask_loss: 0.355 - ETA: 9:12 - loss: 1.1328 - rpn_class_loss: 0.0084 - rpn_bbox_loss: 0.2563 - mrcnn_class_loss: 0.1792 - mrcnn_bbox_loss: 0.3331 - mrcnn_mask_loss: 0.355 - ETA: 9:08 - loss: 1.1321 - rpn_class_loss: 0.0084 - rpn_bbox_loss: 0.2558 - mrcnn_class_loss: 0.1794 - mrcnn_bbox_loss: 0.3330 - mrcnn_mask_loss: 0.355 - ETA: 9:05 - loss: 1.1325 - rpn_class_loss: 0.0084 - rpn_bbox_loss: 0.2555 - mrcnn_class_loss: 0.1799 - mrcnn_bbox_loss: 0.3332 - mrcnn_mask_loss: 0.355 - ETA: 9:00 - loss: 1.1331 - rpn_class_loss: 0.0084 - rpn_bbox_loss: 0.2559 - mrcnn_class_loss: 0.1797 - mrcnn_bbox_loss: 0.3335 - mrcnn_mask_loss: 0.355 - ETA: 8:56 - loss: 1.1325 - rpn_class_loss: 0.0084 - rpn_bbox_loss: 0.2557 - mrcnn_class_loss: 0.1796 - mrcnn_bbox_loss: 0.3332 - mrcnn_mask_loss: 0.355 - ETA: 8:51 - loss: 1.1318 - rpn_cl

371/400 [==========================>...] - ETA: 5:35 - loss: 1.1397 - rpn_class_loss: 0.0086 - rpn_bbox_loss: 0.2585 - mrcnn_class_loss: 0.1812 - mrcnn_bbox_loss: 0.3349 - mrcnn_mask_loss: 0.356 - ETA: 5:31 - loss: 1.1394 - rpn_class_loss: 0.0086 - rpn_bbox_loss: 0.2583 - mrcnn_class_loss: 0.1812 - mrcnn_bbox_loss: 0.3349 - mrcnn_mask_loss: 0.356 - ETA: 5:26 - loss: 1.1396 - rpn_class_loss: 0.0086 - rpn_bbox_loss: 0.2581 - mrcnn_class_loss: 0.1813 - mrcnn_bbox_loss: 0.3351 - mrcnn_mask_loss: 0.356 - ETA: 5:22 - loss: 1.1405 - rpn_class_loss: 0.0087 - rpn_bbox_loss: 0.2578 - mrcnn_class_loss: 0.1822 - mrcnn_bbox_loss: 0.3353 - mrcnn_mask_loss: 0.356 - ETA: 5:18 - loss: 1.1417 - rpn_class_loss: 0.0087 - rpn_bbox_loss: 0.2584 - mrcnn_class_loss: 0.1826 - mrcnn_bbox_loss: 0.3355 - mrcnn_mask_loss: 0.356 - ETA: 5:14 - loss: 1.1418 - rpn_class_loss: 0.0087 - rpn_bbox_loss: 0.2587 - mrcnn_class_loss: 0.1825 - mrcnn_bbox_loss: 0.3354 - mrcnn_mask_loss: 0.356 - ETA: 5:10 - loss: 1.1443 - rpn_cl

400/400 [==============================] - ETA: 1:55 - loss: 1.1427 - rpn_class_loss: 0.0087 - rpn_bbox_loss: 0.2605 - mrcnn_class_loss: 0.1818 - mrcnn_bbox_loss: 0.3352 - mrcnn_mask_loss: 0.356 - ETA: 1:51 - loss: 1.1422 - rpn_class_loss: 0.0087 - rpn_bbox_loss: 0.2602 - mrcnn_class_loss: 0.1817 - mrcnn_bbox_loss: 0.3352 - mrcnn_mask_loss: 0.356 - ETA: 1:47 - loss: 1.1425 - rpn_class_loss: 0.0087 - rpn_bbox_loss: 0.2602 - mrcnn_class_loss: 0.1817 - mrcnn_bbox_loss: 0.3355 - mrcnn_mask_loss: 0.356 - ETA: 1:43 - loss: 1.1423 - rpn_class_loss: 0.0088 - rpn_bbox_loss: 0.2602 - mrcnn_class_loss: 0.1815 - mrcnn_bbox_loss: 0.3354 - mrcnn_mask_loss: 0.356 - ETA: 1:39 - loss: 1.1418 - rpn_class_loss: 0.0087 - rpn_bbox_loss: 0.2598 - mrcnn_class_loss: 0.1817 - mrcnn_bbox_loss: 0.3352 - mrcnn_mask_loss: 0.356 - ETA: 1:35 - loss: 1.1413 - rpn_class_loss: 0.0087 - rpn_bbox_loss: 0.2597 - mrcnn_class_loss: 0.1815 - mrcnn_bbox_loss: 0.3350 - mrcnn_mask_loss: 0.356 - ETA: 1:31 - loss: 1.1411 - rpn_cl

In [2]:
# select trained model 
dir_names = next(os.walk(model.model_dir))[1]
key = config.NAME.lower()
dir_names = filter(lambda f: f.startswith(key), dir_names)
dir_names = sorted(dir_names)

if not dir_names:
    import errno
    raise FileNotFoundError(
            errno.ENOENT,
            "Could not find model directory under {}".format(model.model_dir))          
    
fps = []
# Pick last directory
for d in dir_names: 
    dir_name = os.path.join(model.model_dir, d)
    # Find the last checkpoint
    checkpoints = next(os.walk(dir_name))[2]
    checkpoints = filter(lambda f: f.startswith("mask_rcnn"), checkpoints)
    checkpoints = sorted(checkpoints)
    if not checkpoints:
        print('No weight files in {}'.format(dir_name))
    else: 
        checkpoint = os.path.join(dir_name, checkpoints[-1])
        fps.append(checkpoint)

model_path = sorted(fps)[-1]
print('Found model {}'.format(model_path))

No weight files in E:\学习\ml-lessons\lesson3-data\logs\pneumonia20181023T2038
Found model E:\学习\ml-lessons\lesson3-data\logs\pneumonia20181026T2247\mask_rcnn_pneumonia_0035.h5


In [11]:
model_path = 'E:\学习\ml-lessons\lesson3-data\logs\pneumonia20181026T2247\mask_rcnn_pneumonia_0035.h5'

def iou(box1, box2):
    y11, x11, y12, x12 = box1
    y21, x21, y22, x22 = box2
    
    w1 = x12 - x11
    h1 = y12 - y11
    w2 = x22 - x21
    h2 = y22 - y21
    assert w1 * h1 > 0
    assert w2 * h2 > 0
    
    area1, area2 = w1 * h1, w2 * h2
    xi1, yi1, xi2, yi2 = max([x11, x21]), max([y11, y21]), min([x12, x22]), min([y12, y22])
    
    new_box = [yi1, xi1, yi2, xi2]
    
    if xi2 <= xi1 or yi2 <= yi1:
        return 0, new_box
    else:
        intersect = (xi2-xi1) * (yi2-yi1)
        union = area1 + area2 - intersect
        return intersect / union, new_box

def choose_intersect(model, config, image, threshold=0.2):
    result1 = model.detect([image])
    r1 = result1[0]
    assert( len(r1['rois']) == len(r1['class_ids']) == len(r1['scores']) )
    
    temp_img = np.fliplr(image)
    result2 = model.detect([temp_img])
    r2 = result2[0]
    assert( len(r2['rois']) == len(r2['class_ids']) == len(r2['scores']) )
    
    #r2['rois'] = np.fliplr(r2['rois'])
    num = len(r2['rois'])
    for k in range(num):
        w = r2['rois'][k][3] - r2['rois'][k][1]
        r2['rois'][k][1] = config.IMAGE_MIN_DIM - r2['rois'][k][1] - w
        r2['rois'][k][3] = r2['rois'][k][1] + w
    
    for i, bt in enumerate(r1['rois']):
        max_intersect = 0
        for j, bp in enumerate(r2['rois']):
            intersect, new_box = iou(bt, bp)
            if intersect >= threshold and intersect > max_intersect:
                max_intersect = intersect
                r1['rois'][i] = new_box
                r1['scores'][i] = (r1['scores'][i] + r2['scores'][j]) / 2.0
        if max_intersect == 0:
            r1['scores'][i] = 0
    
    return r1

class InferenceConfig(DetectorConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode='inference', 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

# Get filenames of test dataset DICOM images
test_image_fps = get_dicom_fps(test_dicom_dir)

# Make predictions on test images, write out sample submission 
def predict(image_fps, filepath='submission.csv', min_conf=0.95): 
    
    # assume square image
    resize_factor = ORIG_SIZE / config.IMAGE_SHAPE[0]
    with open(filepath, 'w') as file:
      file.write("patientId,PredictionString\n")
      
      for image_id in tqdm(image_fps): 
        ds = pydicom.read_file(image_id)
        image = ds.pixel_array
        # image = signal.medfilt(image, kernel_size=5)   # 对图片进行中值滤波
        # image = image.astype(np.uint8)
        #image = cv2.equalizeHist(image)  # 对图片进行直方图均衡处理
        #image = sm.closing(image,sm.disk(5))  #用边长为5的圆形滤波器进行闭运算滤波
        image, _ = histep(image)   # 对图片进行直方图均衡处理
          
        # If grayscale. Convert to RGB for consistency.
        if len(image.shape) != 3 or image.shape[2] != 3:
            image = np.stack((image,) * 3, -1) 
        image, window, scale, padding, crop = utils.resize_image(
            image,
            min_dim=config.IMAGE_MIN_DIM,
            min_scale=config.IMAGE_MIN_SCALE,
            max_dim=config.IMAGE_MAX_DIM,
            mode=config.IMAGE_RESIZE_MODE)
            
        patient_id = os.path.splitext(os.path.basename(image_id))[0]
        '''
        results = model.detect([image])
        r = results[0]
        '''
        r=choose_intersect(model,config,image,threshold=0.2)
        
        out_str = ""
        out_str += patient_id 
        out_str += ","
        assert( len(r['rois']) == len(r['class_ids']) == len(r['scores']) )
        if len(r['rois']) == 0: 
            pass
        else: 
            num_instances = len(r['rois'])
            for i in range(num_instances): 
                if r['scores'][i] > min_conf: 
                    out_str += ' '
                    out_str += str(round(r['scores'][i], 2))
                    out_str += ' '

                    # x1, y1, width, height 
                    x1 = r['rois'][i][1]
                    y1 = r['rois'][i][0]
                    width = r['rois'][i][3] - x1 
                    height = r['rois'][i][2] - y1 
                    bboxes_str = "{} {} {} {}".format(x1*resize_factor, y1*resize_factor, \
                                                      width*resize_factor, height*resize_factor)    
                    out_str += bboxes_str

        file.write(out_str+"\n")

sample_submission_fp = 'Mask_RCNN/submission.csv'
predict(test_image_fps, filepath=sample_submission_fp)

Loading weights from  E:\学习\ml-lessons\lesson3-data\logs\pneumonia20181026T2247\mask_rcnn_pneumonia_0035.h5
Re-starting from epoch 35


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [58:14<00:00,  1.16s/it]


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     8
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        3
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.1
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 8
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  64
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  64
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [64 64  3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               3
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           pneumonia
NUM_CLASSES                    2
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST_NMS_ROIS_TRAINING         2000
PRE_NMS_LIMIT                  6000
ROI_POSITIVE_RATIO             0.33
RPN_ANCHOR_RATIOS              [0.5, 1, 2]
RPN_ANCHOR_SCALES              (32, 64)
RPN_ANCHOR_STRIDE              1
RPN_BBOX_STD_DEV               [0.1 0.1 0.2 0.2]
RPN_NMS_THRESHOLD              0.7
RPN_TRAIN_ANCHORS_PER_IMAGE    16
STEPS_PER_EPOCH                100
TOP_DOWN_PYRAMID_SIZE          32
TRAIN_BN                       False
TRAIN_ROIS_PER_IMAGE           16
USE_MINI_MASK                  True
USE_RPN_ROIS                   True
VALIDATION_STEPS               50
WEIGHT_DECAY                   0.0001